In [138]:
import asyncio
from playwright.async_api import async_playwright
import time

async def scrape_links(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        
        links = await page.query_selector_all('a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')
        hrefs = [await link.get_attribute('href') for link in links if await link.get_attribute('href') is not None]
        
        await browser.close()
        return hrefs
hrefs = list()
for i in range(400):
    url = 'https://www.amazon.co.uk/s?k=iphone&i=electronics&rh=n%3A560820%2Cp_72%3A419153031&dc&page=2&qid=1698833966&rnid=419152031&ref=sr_pg_'+str(i+1)
    temp = await scrape_links(url)
    temp = list(set(temp))
    hrefs = hrefs + temp
    time.sleep(2)
hrefs


Future exception was never retrieved
future: <Future finished exception=Error('Connection closed')>
playwright._impl._api_types.Error: Connection closed


CancelledError: 

In [139]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd

async def scrape_amazon_product_details(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
        page = await context.new_page()
        await page.goto(url)
        
        product_title_element = await page.query_selector("#productTitle")
        product_title = await product_title_element.text_content()
        product_title = product_title.strip() if product_title else "Title not found"
        
        product_price_whole_element = await page.query_selector(".a-price-whole")
        product_price_whole = await product_price_whole_element.text_content() if product_price_whole_element else ""
        product_price_decimal_element = await page.query_selector(".a-price-decimal")
        product_price_decimal = await product_price_decimal_element.text_content() if product_price_decimal_element else ""
        product_price = product_price_whole + product_price_decimal
        product_price = product_price.strip() if product_price else "Price not found"
        
        rating_element = await page.query_selector("span[data-hook='rating-out-of-text']")
        rating = await rating_element.text_content() if rating_element else "Rating not found"
        rating = rating.strip() if rating else "Rating not found"
        
        comments_count_element = await page.query_selector("span[data-hook='total-review-count']")
        comments_count = await comments_count_element.text_content() if comments_count_element else "Comments count not found"
        comments_count = comments_count.strip() if comments_count else "Comments count not found"
        
        reviews_link_element = await page.query_selector("a[data-hook='see-all-reviews-link-foot']")
        reviews_link = await reviews_link_element.get_attribute("href") if reviews_link_element else "Reviews link not found"
        reviews_link = "https://www.amazon.co.uk" + reviews_link if reviews_link != "Reviews link not found" else reviews_link
        
        product_details = {}
        table_rows = await page.query_selector_all("tbody > tr")
        for row in table_rows:
            label_element = await row.query_selector("td.a-span3 > span")
            value_element = await row.query_selector("td.a-span9 > span")
            label = await label_element.text_content() if label_element else "Label not found"
            value = await value_element.text_content() if value_element else "Value not found"
            product_details[label.strip()] = value.strip()

        # Creating a dictionary for the product information
        product_info = {
            "Product Title": [product_title][0],
            "Product Price": [product_price][0],
            "Product Rating": [rating][0],
            "Comments Count": [comments_count][0],
            "Reviews Link": [reviews_link][0]
        }
        
        # Combining product details with product information
        product_data = {**product_info, **product_details}
        
        # Creating a DataFrame
        #df = pd.DataFrame(product_data)
        await browser.close()
        
        return product_data

async def scrape_amazon_reviews(reviews_url, product_info):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto(reviews_url)

        review_elements = await page.query_selector_all(".a-section.celwidget")

        reviews_data = []
        for review_element in review_elements:
            review = {}

            
            # Author
            review_author_element = await review_element.query_selector(".a-profile-name")
            review['Review - Author'] = await review_author_element.text_content() if review_author_element else None

            # Rating
            review_rating_element = await review_element.query_selector(".review-rating")
            review['Review - Rating'] = await review_rating_element.text_content() if review_rating_element else None

            # Title
            review_title_element = await review_element.query_selector(".review-title")
            review['Review - Title'] = await review_title_element.text_content() if review_title_element else None

            # Date
            review_date_element = await review_element.query_selector(".review-date")
            review['Review - Date'] = await review_date_element.text_content() if review_date_element else None

            # Review Body
            review_body_element = await review_element.query_selector(".review-text")
            review['Review - Body'] = await review_body_element.text_content() if review_body_element else None

            # Size and Color
            review_format_element = await review_element.query_selector(".review-format-strip")
            review['Review - Format'] = await review_format_element.text_content() if review_format_element else None

            # Merging product info with the review info for each review
            merged_data = {**product_info, **review}
            reviews_data.append(merged_data)

        await browser.close()

    return pd.DataFrame(reviews_data)
for href in range(len(hrefs)):
    print(str(href+1) + " / " + str(len(hrefs)))
    print(hrefs[href])
    url="https://www.amazon.co.uk"+hrefs[href]
    try:
        product_info = await scrape_amazon_product_details(url)
    except:
        continue
    reviews_url = product_info['Reviews Link']
    temp_df = await scrape_amazon_reviews(reviews_url, product_info)
    temp_df['Search'] = 'iPhone'
    df = pd.concat([df, temp_df], ignore_index=True)
    df = df.dropna(subset=['Review - Body']).drop_duplicates(subset=['Review - Body'])
    df.to_csv('amazon_comments.csv', index=False)
df


1 / 4142
/Apple-iPhone-13-128GB-Pink/dp/B09G99YSFG/ref=sr_1_48?keywords=iphone&qid=1698834383&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-48
2 / 4142
/Apple-iPhone-Generation-Black-Renewed/dp/B08D34BNKP/ref=sr_1_30?keywords=iphone&qid=1698834383&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-30
3 / 4142
/Apple-iPhone-Pro-128GB-Silver/dp/B08PCCM1GC/ref=sr_1_27?keywords=iphone&qid=1698834383&refinements=p_72%3A419153031&rnid=419152031&s=telephone&sr=1-27


Future exception was never retrieved
future: <Future finished exception=Error('Connection closed')>
playwright._impl._api_types.Error: Connection closed


Error: net::ERR_ABORTED; maybe frame was detached?
=========================== logs ===========================
navigating to "https://www.amazon.co.uk/Apple-iPhone-Pro-128GB-Silver/product-reviews/B08PCCM1GC/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews", waiting until "load"
============================================================

In [140]:
df

,Product Title,Product Price,Product Rating,Comments Count,Reviews Link,Label not found,Brand,Model name,Operating system,Cellular technology,...,Form factor,Connector type,Front-photo sensor resolution,Speaker type,Special feature,Recommended uses for product,Compatible devices,Mounting type,Unit count,Controller type
0,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone SE 16 GB Smartphone - Space Grey ...,86..,3.5 out of 5,50 global ratings,https://www.amazon.co.uk/Apple-iPhone-SE-Smart...,Value not found,Apple,SE 16GB,IOS 12,"4G, 3G",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,Sonos One (Gen2) The powerful smart speaker wi...,149..,4.7 out of 5,"2,472 global ratings",https://www.amazon.co.uk/Sonos-One-Gen-powerfu...,Value not found,Sonos,Sonos One,NaN,NaN,...,NaN,NaN,NaN,Outdoor,Multi Room Audio,Music,Smartphone,Tabletop,1 Speaker,"App Control, Touchpad Control"
274,Sonos One (Gen2) The powerful smart speaker wi...,149..,4.7 out of 5,"2,472 global ratings",https://www.amazon.co.uk/Sonos-One-Gen-powerfu...,Value not found,Sonos,Sonos One,NaN,NaN,...,NaN,NaN,NaN,Outdoor,Multi Room Audio,Music,Smartphone,Tabletop,1 Speaker,"App Control, Touchpad Control"
275,Sonos One (Gen2) The powerful smart speaker wi...,149..,4.7 out of 5,"2,472 global ratings",https://www.amazon.co.uk/Sonos-One-Gen-powerfu...,Value not found,Sonos,Sonos One,NaN,NaN,...,NaN,NaN,NaN,Outdoor,Multi Room Audio,Music,Smartphone,Tabletop,1 Speaker,"App Control, Touchpad Control"
276,Sonos One (Gen2) The powerful smart speaker wi...,149..,4.7 out of 5,"2,472 global ratings",https://www.amazon.co.uk/Sonos-One-Gen-powerfu...,Value not found,Sonos,Sonos One,NaN,NaN,...,NaN,NaN,NaN,Outdoor,Multi Room Audio,Music,Smartphone,Tabletop,1 Speaker,"App Control, Touchpad Control"
